In [2]:
import numpy as np
import pandas as pd

%xmode Plain
pop=pd.read_csv(r"C:\Users\DAVID SEO\Desktop\state-population.txt")
area=pd.read_csv(r"C:\Users\DAVID SEO\Desktop\state-areas.txt")
abbrevs=pd.read_csv(r"C:\Users\DAVID SEO\Desktop\state-abbrevs.txt")

print(pop.head())
print(area.head())
print(abbrevs.head())





Exception reporting mode: Plain
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


In [9]:
merged=pop.merge(abbrevs,how="outer",left_on="state/region",right_on="abbreviation").drop("abbreviation",axis=1)
cols=['state/region','state', 'ages', 'year', 'population' ]
merged=pd.DataFrame(merged,columns=cols)

'''
Before merging another table. It is a good custom to double-check whether there were any mismatches here,
which we can do by lookinf for rows with nulls.
'''
print(merged.isnull().any())

#This indicates that some of population is null.Note that what we are aming to achieve here is density. 
#Therefore, we should pay attention first to one of elements essential for its calculation.
print(merged[merged.population.isnull()].head())
print()
merged[merged.population.isnull()].tail()

'''
It appear that null values are from PR prior to the year 2000.This is more likely the data not
being available from the original source.

Secondl, we also realizes that some of the state entries are also null, which means there are no 
correspongind entry in the abbrevs key. Let's look at which state lacks this match. 
'''

#Finding the state which contains null values and its corresponding entries in state/region
merged.loc[merged.state.isnull(),"state/region"].unique()

'''
Let's take the code in peices. 
1)merged.state.isnull() returns True if there is null or False if there is a entry. 
2)merged.loc[merged.state.isnull()] returns all the entries if merged.state.isnull() ==True.
3)We want to narrow our scope to the column of 'state.region' 
=>merged.loc[merged.state.isnull(),state.region]
4).unique returns only the unique element 

Before inference, we should be already aware that th state section comes from abbrevs data and state/
region is from population. 

We can infer that the population data  includes state entries for PR and USA while abbreves does not 
have those entries. We canf fix these quickly by filling in appropriate entires
'''
merged.loc[merged["state/region"]=="PR","state"]="Puerto Rico"
merged.loc[merged["state/region"]=="USA","state"]="United States"

# we are now joing the merged with area using a state as a key 
final=merged.merge(area,on="state",how="left")
final.isnull().any()

#From the output, we are aware that threre are some missing entries in area.Let's see which region lacks this inforamtion 
#Also notice from the last section, we have already checked that some information  regarding population is absent prior to year 2012
final.head()


state/region    False
state            True
ages            False
year            False
population       True
dtype: bool
     state/region state     ages  year  population
2448           PR   NaN  under18  1990         NaN
2449           PR   NaN    total  1990         NaN
2450           PR   NaN    total  1991         NaN
2451           PR   NaN  under18  1991         NaN
2452           PR   NaN    total  1993         NaN



,state/region,state,ages,year,population,area (sq. mi)
0,AL,Alabama,under18,2012,1117489.0,52423.0
1,AL,Alabama,total,2012,4817528.0,52423.0
2,AL,Alabama,under18,2010,1130966.0,52423.0
3,AL,Alabama,total,2010,4785570.0,52423.0
4,AL,Alabama,under18,2011,1125763.0,52423.0


In [17]:
final.loc[final["area (sq. mi)"].isnull(),"state"].unique()
'''
The result shows that the DataFrame does not contain the area of the United States as a whole. We could insert the appropriate value
(eg using the sum of all state areas) but in this case we will just drop the null values beacuse the population density of
the Entire United States is not relevant to our current analysis. 
'''
final.dropna(inplace=True)
final.isnull().any() #dobule check if all NaN has been removed. 



state/region     False
state            False
ages             False
year             False
population       False
area (sq. mi)    False
dtype: bool

In [2]:
'''
Aim: rank US states and territories by their 2010 population density.

Now, we have all  the data we need. To answer the quetion of interest, let's first select the protion of data correspoding with
the year 2010 and the total population. We will use the query() function to do quickly 
'''
final.ages.unique() #firt chekc the elemtns of ages : array(['under18', 'total'], dtype=object)

#to give a full information 
data2010=final.query("year==2010 & ages== 'total'")
data2010=data2010.set_index("state")
data2010.head()
data2010["density"]=data2010["population"]/data2010["area (sq. mi)"]
data2010.sort_values(ascending=False,inplace=True,by="density") 

#to giva a partial infromation 
density=data2010["population"]/data2010["area (sq. mi)"]
density.sort_values(ascending=False,inplace=True)
density.head()

NameError: name 'final' is not defined

In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
planets=sns.load_dataset("planets")
'''
Method describe() computes several common aggregates fro each columns and returns the result 
This gives us some insights on the overall properties of a dataset. For example, if you see in the year
colum that alhtough exoplanets were first discovered as far back as 1989, half of all known exoplnets
 were not discovered until 2010 or after. 
'''


'''
Aggregation / Description
count()      total number of items
first(),last()   first and last items
mean,median()  mean and median
min(),max()  minimum and maximum
std(),var()   standard deviation and variance
mad()          mean absolute value
prod()       product of all items
sum()        sum of all items 

These are all methods of DataFrame and Series objects
These are not enough. The next level of data summarization is the groupby oepration, which allows you to
quickly and efficiently compute aggreates on the subsets of data 
'''


###Groupby:Split,Apply,Combine 


'''
- The split step invovles breaking up and grouping a DataFrame depending on the value of specific key
- The apply step invovles computing some function, usually an aggregate,transformation,or filtering,within
  the individual group
- The combine invovles merging the results of these operations in a output array 


While we could certainly do this manually using some combination of the masking,
aggregation, and merging commands covered earlier, it’s important to realize that the
intermediate splits do not need to be explicitly instantiated. Rather, the GroupBy can
(often) do this in a single pass over the data, updating the sum, mean, count, min, or
other aggregate for each group along the way.
'''

df=pd.DataFrame({"key":['A','B','C','A','B','C'],"data":range(6)})
print(df)

#first-taks: We can compute the most basic split-apply-combine operation with the groupby()
#method of DataFrames, passing the name of the desired key column:

df.groupby("key")
'''
that what is returned is not a set of DataFrames, but a DataFrameGroupBy
object.The actual compuation would not appear until the aggreagtion is applied. 
'''
df.groupby("key").sum()



  key  data
0   A     0
1   B     1
2   C     2
3   A     3
4   B     4
5   C     5


,data
key,
A,3
B,5
C,7


In [9]:
'''
method 1 column indexing 
'''
planets.describe() # returns a statistical summary of each column
planets.groupby("method")["orbital_period"] # we have selected a Series group from original group 
                                            #by reference to its column name
#No computation would not occur until we call some aggregate on the object
planets.groupby("method")["orbital_period"].median() 

'''
mtehod 2 iteration over groups. 
The groupby objects also provieds a direct iteration over the groups returing each group as a Series or
DataFrame
'''
for (method,group) in planets.groupby('method'): # method 는  iteration group group은 데이터프레임
    print("{0:30s}shape={1}".format(method,group.shape))

'''
Method 3 Dispatch Methods
:Any method not explicitly implemented by pandas would be passed througha and called on the group. 
(현재까지는 aggregation function을 사용하지 않았다면 그저 내부적으로 그룹별로 묶어서 전달된다고 이해)
Notice these methods are applied to each individual group and the results are combined within Groupby 
and returned. 
'''
planets.groupby("method")["year"].describe().unstack()

'''
method 4 aggreagate,filter,transform,appliy ** higly important concept !!
'''
rng=np.random.RandomState(0)
df=pd.DataFrame({"key":['A','B','C','A','B','C'],
                "data1":range(6),
                "data2":rng.randint(0,10,6)},
               columns=['key','data1','data2'])


df.head()

NameError: name 'planets' is not defined

In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
%xmode Plain
planets=sns.load_dataset("planets")

rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
'data1': range(6),
'data2': rng.randint(0, 10, 6)},
columns = ['key', 'data1', 'data2'])



'''
method 4 1) aggreagate 
aggregate This takes a list form and compute all at once. 
'''
df.groupby('key').aggregate(['min',np.median,max])


'''
method 5 filter operation
drop data based on the group properties
'''
def filter_func(x):
    return x['data2'].std()>4 

print(df)
print(df.groupby('key').std())
print(df.groupby('key').filter(filter_func))

'''
The filter() function should return a Boolean value specifying whether the group
passes the filtering.
'''
def filter_func(x):
    if x>50:
        return True
    else:
        return False

data=planets.loc[planets.method=="Radial Velocity","distance"]
f=filter(filter_func,data)
result=pd.Series(f)



'''
method 6 transformation
transformation can return some transfomred version of the full data to recombine.
'''

planets.groupby("method").transform(lambda x:x-x.mean())
import copy
final=copy.copy(planets)
final["distance"]=final.groupby("method")["distance"].transform(lambda x:x-x.mean())
final[final.distance>0]

'''
method 7 apply method 
The apply() method lets you apply an arbitrary function to the
group results.
'''
def norm_by_data2(x):
    x["data1"]/=x["data2"].sum()
    return x


print(df);print(df.groupby("key").apply(norm_by_data2))


test=copy.copy(planets)
def norma_data(x):
    x["distance"]=(x["distance"]-x["distance"].mean())/x["distance"].std()
    return x

test.groupby("method").apply(norma_data)

#page 168


Exception reporting mode: Plain
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641
  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9
  key     data1  data2
0   A  0.000000      5
1   B  0.142857      0
2   C  0.166667      3
3   A  0.375000      3
4   B  0.571429      7
5   C  0.416667      9


In [19]:
# method 8 A list, array, series, or index providing the grouping keys.
df.groupby(df["key"]).mean() # a series providing the grouping keys:verbose way
df.groupby("key").mean()# simple way we have kept using 


# method 9 A dictionary or series mapping index to group.
df2=df.set_index("key")
mapping={"A":'America',"B":"British","C":"China"}
df2.groupby(mapping).sum()

#method 10 Any Python function.
print(df2)
print(df2.groupby(str.lower).mean())

#method 11 A list of valid keys.
df2.groupby([str.lower,mapping]).mean()


     data1  data2
key              
A        0      5
B        1      0
C        2      3
A        3      3
B        4      7
C        5      9
   data1  data2
a    1.5    4.0
b    2.5    3.5
c    3.5    6.0


,,data1,data2
a,America,1.5,4.0
b,British,2.5,3.5
c,China,3.5,6.0


In [27]:
decade=10*(planets["year"]//10)
decade=decade.astype(str)+'s'
decade.name="decade"
planets.groupby(["method",decade])["number"].sum().unstack("decade").fillna(0)
#page 169

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0
